# Integração CoreML do RusTorch - Kernel Rust

Este notebook demonstra como usar CoreML com RusTorch.
Executa no kernel Rust.

## Verificar Dependências e Recursos Necessários

In [ ]:
// Uso básico do RusTorch
extern crate rustorch;

use rustorch::tensor::Tensor;
use rustorch::gpu::DeviceType;

println!("Versão do RusTorch: {}", env!("CARGO_PKG_VERSION"));
println!("Versão do Rust: {}", env!("RUSTC_VERSION"));

## Verificar Disponibilidade do CoreML

Verificar se CoreML está disponível no sistema atual.

In [ ]:
#[cfg(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback"))]
{
    use rustorch::backends::DeviceManager;
    
    let coreml_available = DeviceManager::is_coreml_available();
    println!("CoreML disponível: {}", coreml_available);
    
    if coreml_available {
        println!("🎉 CoreML está disponível!");
        println!("Plataforma: macOS");
        
        // Exibir informações do dispositivo
        use rustorch::gpu::coreml::device_cache::DeviceCache;
        let cache = DeviceCache::global();
        cache.warmup();
        
        let stats = cache.get_stats();
        println!("Estatísticas do cache: {:?}", stats);
    } else {
        println!("⚠️ CoreML não está disponível");
        println!("Por favor, use CPU ou outros backends GPU");
    }
}

#[cfg(not(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback")))]
{
    println!("❌ Recursos CoreML não estão habilitados");
    println!("Por favor, compile com --features coreml");
}

## Operações Básicas de Tensor

In [ ]:
// Criar tensores básicos
let a = Tensor::zeros(&[2, 3]);
let b = Tensor::ones(&[3, 2]);

println!("Forma do tensor A: {:?}", a.shape());
println!("Forma do tensor B: {:?}", b.shape());

// Multiplicação básica de matrizes
let result = a.matmul(&b);
println!("Forma do resultado: {:?}", result.shape());
println!("Operações básicas de tensor completadas");

## Operações com Dispositivo CoreML

In [ ]:
#[cfg(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback"))]
{
    use rustorch::gpu::coreml::{CoreMLDevice, CoreMLBackend};
    use rustorch::backends::BackendConfig;
    
    // Tentar criar dispositivo CoreML
    match CoreMLDevice::new(0) {
        Ok(device) => {
            println!("🖥️ Dispositivo CoreML criado com sucesso");
            println!("ID do dispositivo: {}", device.id());
            println!("Disponível: {}", device.is_available());
            println!("Limite de memória: {} MB", device.memory_limit() / (1024 * 1024));
            
            // Criar configuração do backend
            let config = BackendConfig::new()
                .with_caching(true)
                .with_max_cache_size(200)
                .with_profiling(true)
                .with_auto_fallback(true);
            
            println!("⚙️ Configuração do backend: {:?}", config);
            
            // Criar backend CoreML
            match CoreMLBackend::new(device, config) {
                Ok(backend) => {
                    println!("🚀 Backend CoreML inicializado");
                    
                    // Obter estatísticas
                    let stats = backend.get_statistics();
                    println!("📊 Estatísticas do backend:");
                    println!("  Operações totais: {}", stats.total_operations);
                    println!("  Acertos de cache: {}", stats.cache_hits);
                    println!("  Erros de cache: {}", stats.cache_misses);
                    println!("  Operações de fallback: {}", stats.fallback_operations);
                    
                    // Criar tensores no CoreML
                    let tensor_a = Tensor::randn(&[64, 64]).to_device(&backend);
                    let tensor_b = Tensor::randn(&[64, 64]).to_device(&backend);
                    
                    println!("📐 Tensores criados no dispositivo CoreML");
                    
                    // Operação de multiplicação de matrizes
                    let start = std::time::Instant::now();
                    let result = tensor_a.matmul(&tensor_b);
                    let duration = start.elapsed();
                    
                    println!("✅ Multiplicação de matrizes completada");
                    println!("⏱️ Tempo de execução: {:?}", duration);
                    println!("🎯 Forma do resultado: {:?}", result.shape());
                    
                    // Limpar cache
                    backend.cleanup_cache();
                    println!("🧹 Cache limpo");
                }
                Err(e) => println!("❌ Erro ao criar backend CoreML: {:?}", e),
            }
        }
        Err(e) => {
            println!("❌ Erro ao criar dispositivo CoreML: {:?}", e);
            println!("CoreML pode não estar disponível neste sistema");
        }
    }
}

#[cfg(not(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback")))]
{
    println!("⚠️ Pulando operações CoreML - recursos não habilitados");
}

## Comparação de Performance

In [ ]:
use std::time::Instant;

fn benchmark_operations() {
    let sizes = vec![(64, 64), (128, 128), (256, 256), (512, 512)];
    
    println!("🏁 Benchmarking de operações:");
    println!("Tamanho\t\tCPU (ms)\tDispositivo Preferido");
    println!("-" * 48);
    
    for (rows, cols) in sizes {
        // Criar tensores na CPU
        let a = Tensor::randn(&[rows, cols]);
        let b = Tensor::randn(&[cols, rows]);
        
        // Medir tempo da CPU
        let start = Instant::now();
        let _result = a.matmul(&b);
        let cpu_duration = start.elapsed();
        
        // Determinar dispositivo preferido
        let preferred_device = if rows * cols < 1000 {
            "CPU"
        } else if rows * cols < 10000 {
            "Metal GPU"
        } else {
            "CoreML"
        };
        
        println!("{}x{}\t\t{:.2}\t\t{}", 
                rows, cols, 
                cpu_duration.as_millis() as f64, 
                preferred_device);
    }
}

benchmark_operations();
println!("\n📝 Nota: A seleção de dispositivo é baseada no tamanho do tensor e disponibilidade");

## Seleção Inteligente de Dispositivo

In [ ]:
#[cfg(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback"))]
{
    use rustorch::backends::{DeviceManager, DeviceSelector};
    
    fn demonstrate_device_selection() {
        println!("🎯 Demonstração de seleção inteligente de dispositivo:");
        
        let operations = vec![
            ("Multiplicação pequena", vec![16, 16], "CPU"),
            ("Convolução 2D", vec![32, 3, 224, 224], "CoreML"),
            ("Transformação de matriz", vec![128, 128], "Metal GPU"),
            ("Operação de lote grande", vec![512, 512], "CoreML"),
            ("Cálculo vetorial", vec![1000], "CPU"),
        ];
        
        for (name, shape, preferred) in operations {
            println!("  {:<25} {:?} -> {}", name, shape, preferred);
            
            // Simular seleção baseada em regras
            let tensor_size: usize = shape.iter().product();
            let selected_device = match tensor_size {
                size if size < 1000 => DeviceType::Cpu,
                size if size < 50000 => DeviceType::MetalGpu,
                _ => {
                    if DeviceManager::is_coreml_available() {
                        DeviceType::CoreML
                    } else {
                        DeviceType::MetalGpu
                    }
                }
            };
            
            println!("    -> Dispositivo selecionado: {:?}", selected_device);
        }
        
        println!("\n📝 Lógica de seleção:");
        println!("  • < 1K elementos: CPU (overhead mínimo)");
        println!("  • 1K-50K elementos: Metal GPU (balanceado)");
        println!("  • > 50K elementos: CoreML (otimizado) ou Metal GPU (fallback)");
    }
    
    demonstrate_device_selection();
}

#[cfg(not(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback")))]
{
    println!("⚠️ Demonstração de seleção de dispositivo pulada - recursos CoreML não disponíveis");
}

## Exemplo Avançado: Camada de Rede Neural

In [ ]:
fn simulate_neural_layer() {
    println!("🧠 Simulação de camada de rede neural:");
    
    // Configuração da camada
    let batch_size = 32;
    let input_dim = 784;   // 28x28 MNIST
    let hidden_dim = 256;
    let output_dim = 10;   // 10 classes
    
    println!("📊 Configuração:");
    println!("  Tamanho do lote: {}", batch_size);
    println!("  Dimensão de entrada: {}", input_dim);
    println!("  Dimensão oculta: {}", hidden_dim);
    println!("  Dimensão de saída: {}", output_dim);
    
    // Criar tensores
    let input = Tensor::randn(&[batch_size, input_dim]);
    let weight1 = Tensor::randn(&[input_dim, hidden_dim]);
    let weight2 = Tensor::randn(&[hidden_dim, output_dim]);
    
    println!("\n🔄 Forward pass:");
    
    // Forward pass simulado
    let start = Instant::now();
    
    // Camada 1: entrada -> oculta
    let hidden = input.matmul(&weight1);
    println!("  ✅ Entrada -> Oculta: {:?}", hidden.shape());
    
    // Função de ativação ReLU (simulada)
    let activated = hidden.relu();
    println!("  ✅ Ativação ReLU aplicada");
    
    // Camada 2: oculta -> saída
    let output = activated.matmul(&weight2);
    println!("  ✅ Oculta -> Saída: {:?}", output.shape());
    
    let total_time = start.elapsed();
    
    println!("\n⏱️ Tempo total do forward pass: {:?}", total_time);
    println!("🚀 Performance estimada: {:.0} amostras/segundo", 
             (batch_size as f64) / total_time.as_secs_f64());
    
    println!("\n📝 Em uma implementação real:");
    println!("  • Matrizes grandes usariam CoreML");
    println!("  • Ativações usariam Metal GPU");
    println!("  • Operações pequenas permaneceriam na CPU");
}

simulate_neural_layer();

## Tratamento de Erros e Fallback

In [ ]:
fn demonstrate_fallback_behavior() {
    println!("🔄 Demonstração de comportamento de fallback:");
    
    // Simular operação que pode falhar no CoreML
    let complex_tensor = Tensor::randn(&[100, 100]);
    
    println!("🎯 Tentando operação CoreML...");
    
    // Na implementação real, isso seria:
    // match tensor.to_coreml() {
    //     Ok(coreml_tensor) => { /* usar CoreML */ },
    //     Err(_) => { /* fallback para Metal/CPU */ }
    // }
    
    let use_coreml = false; // Simular falha do CoreML
    
    if use_coreml {
        println!("✅ Operação CoreML bem-sucedida");
    } else {
        println!("⚠️ CoreML não disponível, usando fallback");
        
        // Fallback para Metal GPU
        let start = Instant::now();
        let result = complex_tensor.matmul(&complex_tensor);
        let fallback_time = start.elapsed();
        
        println!("✅ Operação de fallback completada");
        println!("⏱️ Tempo de fallback: {:?}", fallback_time);
        println!("📐 Forma do resultado: {:?}", result.shape());
    }
    
    println!("\n📝 Estratégia de fallback:");
    println!("  1. Tentar CoreML (melhor performance)");
    println!("  2. Fallback para Metal GPU (boa compatibilidade)");
    println!("  3. Fallback final para CPU (máxima compatibilidade)");
}

demonstrate_fallback_behavior();

## Resumo e Próximos Passos

In [ ]:
println!("📋 Resumo da Integração CoreML do RusTorch (Kernel Rust):");
println!();
println!("✅ Recursos demonstrados:");
println!("  • Verificação de disponibilidade CoreML");
println!("  • Criação e gerenciamento de dispositivos");
println!("  • Configuração do backend");
println!("  • Operações básicas de tensor");
println!("  • Benchmarking de performance");
println!("  • Seleção inteligente de dispositivo");
println!("  • Comportamento de fallback");
println!();
println!("🚧 Área de desenvolvimento:");
println!("  • Implementação completa de operações CoreML");
println!("  • Otimização de transferência de memória");
println!("  • Suporte estendido para tipos de tensor");
println!("  • Profiling detalhado de performance");
println!("  • Integração com pipelines ML");
println!();
println!("🎯 Próximos passos recomendados:");
println!("  1. Testar com modelos CoreML pré-treinados");
println!("  2. Benchmark comparativo com outros backends");
println!("  3. Otimizar para casos de uso específicos");
println!("  4. Implantar em aplicações de produção");
println!();

#[cfg(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback"))]
{
    if rustorch::backends::DeviceManager::is_coreml_available() {
        println!("🎉 Todos os recursos CoreML estão disponíveis para teste!");
    } else {
        println!("⚠️ CoreML está habilitado mas não disponível neste sistema");
    }
}

#[cfg(not(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback")))]
{
    println!("⚠️ Compile com recursos CoreML para funcionalidade completa");
}

println!("\n🚀 Pronto para desenvolvimento avançado CoreML com RusTorch!");